# Análises de Eficiência Computacional

## Configurações Iniciais

In [0]:
%pip install mlflow==3.6.0 xgboost lightgbm catboost graphviz==0.21 memory_profiler
dbutils.library.restartPython()

In [0]:
import time
import psutil
import pandas as pd
import numpy as np
from datetime import datetime
import mlflow
import warnings
warnings.filterwarnings('ignore')

In [0]:
X_train_scaled = spark.read.table("my_catalog.default.creditcard_x_train_scaled_tcc").toPandas()
X_test_scaled = spark.read.table("my_catalog.default.creditcard_x_test_scaled_tcc").toPandas()
y_train = spark.read.table("my_catalog.default.creditcard_train_pdd_tcc").toPandas()["Class"]
y_test = spark.read.table("my_catalog.default.creditcard_test_pd_tcc").toPandas()["Class"]

In [0]:
model_uris = {
    "XGBoost": "models:/workspace.default.creditcard-fraud-xgboost-optimized/1",
    "RandomForest": "models:/workspace.default.creditcard-fraud-randomforest-smote/1",
    "CatBoost": "models:/workspace.default.creditcard-fraud-catboost-optimized/1",
    "LightGBM": "models:/workspace.default.creditcard-fraud-lightgbm-smote/1",
    "MLP_Classifier": "models:/workspace.default.creditcard-fraud-mlp_classifier-optimized/1",
    "DecisionTree": "models:/workspace.default.creditcard-fraud-decisiontree-optimized/1",
    "Logistic_Regression": "models:/workspace.default.creditcard-logistic-regression-optimized/1"    
}
models = {}
for model_name, model_uri in model_uris.items():
    try:
        model = mlflow.pyfunc.load_model(model_uri)
        models[model_name] = model
        print(f"Modelo {model_name} carregado com sucesso")
    except Exception as e:
        print(f"Erro ao carregar {model_name}: {e}")

In [0]:
def load_mlflow_metrics(model_uris):
    """carrega as métricas armazenadas no MLflow Registry"""
    mlflow_metrics = []
    client = mlflow.tracking.MlflowClient()
    
    for model_name, model_uri in model_uris.items():
        try:
            model_version = model_uri.split('/')[-1]
            model_path = model_uri.replace(f"/{model_version}", "")
            model_name_registry = model_path.split('/')[-1]
            
            
            versions = client.search_model_versions(f"name='{model_name_registry}'")
            
            version_obj = next((v for v in versions if str(v.version) == str(model_version)), None)
            
            if version_obj and getattr(version_obj, "run_id", None):
                run_id = version_obj.run_id
                run = client.get_run(run_id)
                metrics = run.data.metrics
                mlflow_metrics.append({
                    'model': model_name,
                    'roc_auc': metrics.get('test_roc_auc', metrics.get('roc_auc', 0)),
                    'pr_auc': metrics.get('test_avg_precision', metrics.get('avg_precision', 0)),
                    'precision': metrics.get('test_precision', metrics.get('precision', 0)),
                    'recall': metrics.get('test_recall', metrics.get('recall', 0)),
                    'f1_score': metrics.get('test_f1', metrics.get('f1_score', 0)),
                    'source': 'mlflow'
                })
                print(f"Métricas carregadas do MLflow para {model_name}")
            else:
                
                print(f"Erro ao carregar métricas do MLflow para {model_name}: run_id não encontrado para versão {model_version}. Verifique se o modelo foi registrado corretamente no MLflow Model Registry e se a versão existe.")
        except Exception as e:
            print(f"Erro ao carregar métricas do MLflow para {model_name}: {e}")
    
    return pd.DataFrame(mlflow_metrics)
try:
    mlflow_predictive_results = load_mlflow_metrics(model_uris)
    print("\nMétricas do MLflow carregadas com sucesso")
except Exception as e:
    print(f"Não foi possível carregar métricas do MLflow: {e}")
    mlflow_predictive_results = pd.DataFrame()

In [0]:
print("Resultados preditivos carregados do MLflow")
display(mlflow_predictive_results)
all_predictive_results = mlflow_predictive_results.copy()

## Scores

In [0]:
weights = {
    "roc_auc": 0.2,
    "avg_precision": 0.25,
    "precision": 0.15,
    "recall": 0.2,
    "f1_score": 0.2
}

def calculate_weighted_score(row, weights):
    metrics = {
        "roc_auc": row.get("roc_auc", 0),
        "avg_precision": row.get("pr_auc", 0),
        "precision": row.get("precision", 0),
        "recall": row.get("recall", 0),
        "f1_score": row.get("f1_score", 0)
    }
    normalized_metrics = {k: (v if v is not None else 0) for k, v in metrics.items()}
    return sum(normalized_metrics[k] * weights[k] for k in metrics.keys())

all_predictive_results["weighted_score"] = all_predictive_results.apply(lambda row: calculate_weighted_score(row, weights), axis=1)

print("Análise das métricas dos modelos:")
for idx, row in all_predictive_results.iterrows():
    print(f"\nModelo: {row['model']}")
    print(f"roc_auc: {row['roc_auc']:.4f}")
    print(f"avg_precision: {row['pr_auc']:.4f}")
    print(f"precision: {row['precision']:.4f}")
    print(f"recall: {row['recall']:.4f}")
    print(f"f1_score: {row['f1_score']:.4f}")
    print(f"Score combinado para problemas desbalanceados: {row['weighted_score']:.4f}")

display(all_predictive_results)

| Model              | ROC AUC   | PR AUC    | Precision  | Recall    | F1-Score  | Source | Weighted Score |
|--------------------|-----------|-----------|------------|-----------|-----------|--------|----------------|
| XGBoost            | 0.9585    | 0.8145    | 0.9383     | 0.7755    | 0.8492    | mlflow | 0.8610         |
| RandomForest       | 0.9660    | 0.8105    | 0.9059     | 0.7857    | 0.8415    | mlflow | 0.8572         |
| CatBoost           | 0.9787    | 0.7721    | 0.9259     | 0.7653    | 0.8380    | mlflow | 0.8483         |
| LightGBM           | 0.9698    | 0.7830    | 0.8387     | 0.7959    | 0.8168    | mlflow | 0.8381         |
| MLP_Classifier     | 0.9754    | 0.7657    | 0.8941     | 0.7755    | 0.8306    | mlflow | 0.8418         |
| DecisionTree       | 0.8978    | 0.7321    | 0.8837     | 0.7755    | 0.8261    | mlflow | 0.8155         |
| Logistic_Regression| 0.9683    | 0.7357    | 0.8523     | 0.7653    | 0.8065    | mlflow | 0.8198         |

In [0]:
import matplotlib.pyplot as plt

stacking_row = pd.DataFrame([{
    "model": "Stacking_Ensemble",
    "weighted_score": 0.8425
}])
sorted_df = pd.concat([all_predictive_results, stacking_row], ignore_index=True).sort_values("weighted_score", ascending=False)

plt.figure(figsize=(10, 6))
bars = plt.bar(sorted_df["model"], sorted_df["weighted_score"], color='#ffcccc', edgecolor='none')
plt.title("\nScore de Desempenho Preditivo por Modelo\n", fontweight='bold')
plt.xticks(rotation=45)
plt.tight_layout()

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().yaxis.set_ticks([])

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{height:.4f}", ha='center', va='bottom', fontsize=10)

plt.show()

In [0]:
import time
import psutil
import os

efficiency_results = []

for model_name, model in models.items():
    start_time = time.time()
    y_pred = model.predict(X_test_scaled)
    inference_time = time.time() - start_time

    process = psutil.Process(os.getpid())
    memory_usage_mb = process.memory_info().rss / (1024 * 1024)
    cpu_percent = psutil.cpu_percent(interval=1)
    num_threads = process.num_threads()

    efficiency_results.append({
        "model": model_name,
        "inference_time_sec": inference_time,
        "memory_usage_mb": memory_usage_mb,
        "cpu_percent": cpu_percent,
        "num_threads": num_threads
    })

    mlflow.log_metric(f"{model_name}_inference_time_sec", inference_time)
    mlflow.log_metric(f"{model_name}_memory_usage_mb", memory_usage_mb)
    mlflow.log_metric(f"{model_name}_cpu_percent", cpu_percent)
    mlflow.log_metric(f"{model_name}_num_threads", num_threads)

efficiency_df = pd.DataFrame(efficiency_results)
display(efficiency_df)

### Com 10 runs, pegando a média dos runs.

In [0]:
import time
import psutil
import os

n_runs = 10

def run_efficiency_test(model, X):
    inference_times = []
    memory_usages = []
    cpu_percents = []
    num_threads_list = []

    for _ in range(n_runs):
        start_time = time.time()
        _ = model.predict(X)
        inference_time = time.time() - start_time
        inference_times.append(inference_time)

        process = psutil.Process(os.getpid())
        memory_usages.append(process.memory_info().rss / (1024 * 1024))
        cpu_percents.append(psutil.cpu_percent(interval=0.1))
        num_threads_list.append(process.num_threads())

    return {
        "inference_time_sec": np.mean(inference_times),
        "memory_usage_mb": np.mean(memory_usages),
        "cpu_percent": np.mean(cpu_percents),
        "num_threads": np.mean(num_threads_list)
    }

efficiency_results = []

for model_name, model in models.items():
    metrics = run_efficiency_test(model, X_test_scaled)
    metrics["model"] = model_name

    mlflow.log_metric(f"{model_name}_inference_time_sec", metrics["inference_time_sec"])
    mlflow.log_metric(f"{model_name}_memory_usage_mb", metrics["memory_usage_mb"])
    mlflow.log_metric(f"{model_name}_cpu_percent", metrics["cpu_percent"])
    mlflow.log_metric(f"{model_name}_num_threads", metrics["num_threads"])

    efficiency_results.append(metrics)

efficiency_df_10runs = pd.DataFrame(efficiency_results)
display(efficiency_df_10runs)

In [0]:
# valores de normalização e pesos dados as métricas podem e devem ser alterados de acordo com o caso e o que se deseja medir
# definimos os valores abaixo considerando o contexto de detecção de fraude em cartão de crédito, onde é necessário equilibrar tempo de resposta rápido (inferência < 5s), uso moderado de memória (< 1GB), baixo consumo de CPU (< 100%) e número de threads (< 16), priorizando agilidade e escalabilidade em ambiente produtivo.

# normalização dos valores de eficiência computacional
max_inference_time = 5.0  # segundos
max_memory_usage = 1024.0  # MB
max_cpu_percent = 100.0
max_num_threads = 16.0

def compute_efficiency_score(row):
    inference_time = row.get("inference_time_sec", 0)
    memory_usage_mb = row.get("memory_usage_mb", 0)
    cpu_percent = row.get("cpu_percent", 0)
    num_threads = row.get("num_threads", 0)
    score = (
        (1 - min(inference_time / max_inference_time, 1)) * 0.4 +
        (1 - min(memory_usage_mb / max_memory_usage, 1)) * 0.4 +
        (1 - min(cpu_percent / max_cpu_percent, 1)) * 0.1 +
        (1 - min(num_threads / max_num_threads, 1)) * 0.1
    )
    return score

efficiency_df_10runs["efficiency_score"] = efficiency_df_10runs.apply(compute_efficiency_score, axis=1)

# score final de trade-off entre desempenho preditivo e eficiência computacional
# pesos: 0.7 para weighted_score, 0.3 para efficiency_score

merged_df = pd.merge(all_predictive_results, efficiency_df_10runs, on="model")
merged_df["overall_score"] = merged_df["weighted_score"] * 0.7 + merged_df["efficiency_score"] * 0.3

for idx, row in merged_df.iterrows():
    print(f"\nModelo: {row['model']}")
    print(f"Score de eficiência computacional: {row['efficiency_score']:.4f}")
    print(f"Score combinado (trade-off): {row['overall_score']:.4f}")
    mlflow.log_metric(f"{row['model']}_efficiency_score", row['efficiency_score'])
    mlflow.log_metric(f"{row['model']}_overall_score", row['overall_score'])

display(merged_df)

In [0]:
#from pyspark.sql import Row

# cria lista de dicionários com todas as métricas dos modelos
metrics_rows = []
for idx, row in merged_df.iterrows():
    metrics_rows.append(Row(
        model=row['model'],
        roc_auc=row['roc_auc'],
        pr_auc=row['pr_auc'],
        precision=row['precision'],
        recall=row['recall'],
        f1_score=row['f1_score'],
        weighted_score=row['weighted_score'],
        inference_time_sec=row['inference_time_sec'],
        memory_usage_mb=row['memory_usage_mb'],
        cpu_percent=row['cpu_percent'],
        num_threads=row['num_threads'],
        efficiency_score=row['efficiency_score'],
        overall_score=row['overall_score'],
        source=row.get('source', 'mlflow')
    ))


metrics_models_spark_df = spark.createDataFrame(metrics_rows)

metrics_models_spark_df.write.mode("overwrite").saveAsTable("my_catalog.default.final_model_metrics_all_10_runs")

In [0]:
metrics_models_df_10_runs = spark.table("my_catalog.default.final_model_metrics_all_10_runs")
display(metrics_models_df_10_runs)

In [0]:
metrics_stacking_df_10_runs = spark.table("my_catalog.default.stacking_final_metrics_10_runs")
display(metrics_stacking_df_10_runs)

In [0]:
stacking_perf_row = metrics_stacking_df_10_runs.filter(metrics_stacking_df_10_runs["model"] == "StackingEnsemble").toPandas().iloc[0]

base_models = ["XGBoost", "RandomForest", "CatBoost", "LightGBM", "MLP_Classifier"]
base_metrics = efficiency_df_10runs[efficiency_df_10runs["model"].isin(base_models)].copy()

total_inference_time = base_metrics["inference_time_sec"].sum() + stacking_perf_row["inference_time_sec"]
max_memory_usage = max(base_metrics["memory_usage_mb"].max(), stacking_perf_row["memory_usage_mb"])
max_num_threads = max(base_metrics["num_threads"].max(), stacking_perf_row["num_threads"])
max_cpu_percent = max(base_metrics["cpu_percent"].max(), stacking_perf_row["cpu_percent"])

max_inference_time = 5.0
max_memory_usage_norm = 1024.0
max_cpu_percent_norm = 100.0
max_num_threads_norm = 16.0

def compute_efficiency_score(row):
    score = (
        (1 - min(row["inference_time_sec"] / max_inference_time, 1)) * 0.4 +
        (1 - min(row["memory_usage_mb"] / max_memory_usage_norm, 1)) * 0.4 +
        (1 - min(row["cpu_percent"] / max_cpu_percent_norm, 1)) * 0.1 +
        (1 - min(row["num_threads"] / max_num_threads_norm, 1)) * 0.1
    )
    return score

stacking_efficiency_score = compute_efficiency_score({
    "inference_time_sec": total_inference_time,
    "memory_usage_mb": max_memory_usage,
    "cpu_percent": max_cpu_percent,
    "num_threads": max_num_threads
})

stacking_weighted_score = stacking_perf_row["weighted_score"]
precision = stacking_perf_row["precision"]
recall = stacking_perf_row["recall"]
stacking_pr_auc = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
stacking_overall_score = stacking_weighted_score * 0.7 + stacking_efficiency_score * 0.3

stacking_final_row = pd.DataFrame([{
    "model": "StackingEnsemble",
    "roc_auc": stacking_perf_row["roc_auc"],
    "pr_auc": stacking_pr_auc,
    "precision": precision,
    "recall": recall,
    "f1_score": stacking_perf_row["f1_score"],
    "weighted_score": stacking_weighted_score,
    "inference_time_sec": total_inference_time,
    "memory_usage_mb": max_memory_usage,
    "cpu_percent": max_cpu_percent,
    "num_threads": max_num_threads,
    "efficiency_score": stacking_efficiency_score,
    "overall_score": stacking_overall_score,
    "source": stacking_perf_row.get("source", "unity_catalog")
}])

merged_df_with_stacking_10_runs = pd.concat([merged_df, stacking_final_row], ignore_index=True)
merged_df_with_stacking_10_runs = merged_df_with_stacking_10_runs.sort_values("overall_score", ascending=False).reset_index(drop=True)
display(merged_df_with_stacking_10_runs)

In [0]:
merged_df_with_stacking_10_runs = pd.concat([merged_df, stacking_final_row], ignore_index=True)
merged_df_with_stacking_10_runs = merged_df_with_stacking.sort_values("overall_score", ascending=False).reset_index(drop=True)
display(merged_df_with_stacking_10_runs)

spark_df_with_stacking_10_runs = spark.createDataFrame(merged_df_with_stacking_10_runs)
spark_df_with_stacking_10_runs.write.mode("overwrite").saveAsTable("my_catalog.default.final_model_metrics_with_stacking_10_runs")

In [0]:
#import matplotlib.pyplot as plt

df = spark_df_with_stacking_10_runs.toPandas()
df = df.sort_values("efficiency_score", ascending=False)
plt.figure(figsize=(10, 6))
bars = plt.bar(df["model"], df["efficiency_score"], color='#cceeff', edgecolor='none')
plt.title("\nScore de Eficiência Computacional por Modelo no Volume Base (56.961 transações):\n", fontweight='bold')
plt.xlabel("Modelo")
plt.xticks(rotation=45)
plt.tight_layout()

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().set_yticks([])
plt.gca().set_ylabel("") 

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{height:.4f}", ha='center', va='bottom', fontsize=10)

plt.show()

| Ranking | Modelo            | Tempo de Inferência (s) | Uso de Memória (MB) | Uso de CPU (%) | Nº Threads | Score Ef. Computacional |
|---------|-------------------|-----------|--------------|---------|---------|------------------|
| 1º      | CatBoost          | 0.094     | 699.79       | 16.35   | 30      | 0.603            |
| 2º      | DecisionTree      | 0.022     | 701.80       | 24.25   | 30      | 0.600            |
| 3º      | LightGBM          | 0.147     | 700.38       | 20.35   | 30      | 0.594            |
| 4º      | LogisticRegression| 0.022     | 701.80       | 56.75   | 30      | 0.567            |
| 5º      | RandomForest      | 0.560     | 699.65       | 22.45   | 30      | 0.559            |
| 6º      | XGBoost           | 0.752     | 699.44       | 25.45   | 30      | 0.541            |
| 7º      | MLP_Classifier    | 0.581     | 701.80       | 43.25   | 30      | 0.536            |
| 8º      | StackingEnsemble  | 2.136     | 701.80       | 43.25   | 40      | 0.412            |

## Simulação de sobrecarga de dados para todos os modelos

In [0]:
original_count = len(X_test_scaled)
display(original_count)

In [0]:
# simulação de sobrecarga de dadospara todos os modelos
factors = [10, 50, 100]  # fatores a testar
n_runs = 10  #  10 execuções por fator

original_count = len(X_test_scaled)

for factor in factors:
    print(f"\n--- Testando com fator {factor}) ---")
    bigdata_X_test = pd.concat([X_test_scaled] * factor, ignore_index=True)
    bigdata_y_test = np.tile(y_test, factor)

    efficiency_results_big = []

    for model_name, model in models.items():
        inference_times_big = []
        memory_usages_big = []
        cpu_percents_big = []
        num_threads_list_big = []

        for _ in range(n_runs):
            start_time = time.time()
            _ = model.predict(bigdata_X_test)
            inference_time = time.time() - start_time
            inference_times_big.append(inference_time)

            process = psutil.Process(os.getpid())
            memory_usages_big.append(process.memory_info().rss / (1024 * 1024))
            cpu_percents_big.append(psutil.cpu_percent(interval=0.1))
            num_threads_list_big.append(process.num_threads())

        metrics_big = {
            "model": f"{model_name}_BigData_Factor{factor}",
            "inference_time_sec": np.mean(inference_times_big),
            "memory_usage_mb": np.mean(memory_usages_big),
            "cpu_percent": np.mean(cpu_percents_big),
            "num_threads": np.mean(num_threads_list_big),
            "factor": factor,
            "total_records": original_count * factor
        }

        mlflow.log_metric(f"{model_name}_inference_time_sec_bigdata_factor{factor}", metrics_big["inference_time_sec"])
        mlflow.log_metric(f"{model_name}_memory_usage_mb_bigdata_factor{factor}", metrics_big["memory_usage_mb"])
        mlflow.log_metric(f"{model_name}_cpu_percent_bigdata_factor{factor}", metrics_big["cpu_percent"])
        mlflow.log_metric(f"{model_name}_num_threads_bigdata_factor{factor}", metrics_big["num_threads"])

        efficiency_results_big.append(metrics_big)

    efficiency_df_bigdata = pd.DataFrame(efficiency_results_big)
    display(efficiency_df_bigdata)

In [0]:
# salva resultados dos testes de eficiência dos modelos com fator 10, 50 e 100 no Unity Catalog
for factor in factors:
    bigdata_X_test = pd.concat([X_test_scaled] * factor, ignore_index=True)
    bigdata_y_test = np.tile(y_test, factor)

    efficiency_results_big = []
    for model_name, model in models.items():
        inference_times_big = []
        memory_usages_big = []
        cpu_percents_big = []
        num_threads_list_big = []

        for _ in range(n_runs):
            start_time = time.time()
            _ = model.predict(bigdata_X_test)
            inference_time = time.time() - start_time
            inference_times_big.append(inference_time)

            process = psutil.Process(os.getpid())
            memory_usages_big.append(process.memory_info().rss / (1024 * 1024))
            cpu_percents_big.append(psutil.cpu_percent(interval=0.1))
            num_threads_list_big.append(process.num_threads())

        metrics_big = {
            "model": f"{model_name}_BigData_Factor{factor}",
            "inference_time_sec": np.mean(inference_times_big),
            "memory_usage_mb": np.mean(memory_usages_big),
            "cpu_percent": np.mean(cpu_percents_big),
            "num_threads": np.mean(num_threads_list_big),
            "factor": factor,
            "total_records": len(bigdata_X_test)
        }
        efficiency_results_big.append(metrics_big)

    efficiency_df_bigdata = pd.DataFrame(efficiency_results_big)
    spark_df_bigdata = spark.createDataFrame(efficiency_df_bigdata)
    spark_df_bigdata.write.mode("append").saveAsTable("my_catalog.default.model_efficiency_bigdata_tests")

In [0]:
#carregar dataset com stack para fator 10,50,100.
metrics_bigdata_stacking_df = spark.table("my_catalog.default.stacking_bigdata_metrics")
display(metrics_bigdata_stacking_df)
metrics_bigdata_all_models_df = spark.table("my_catalog.default.model_efficiency_bigdata_tests")
display(metrics_bigdata_all_models_df)

In [0]:
base_models = ["XGBoost_BigData_Factor10", "RandomForest_BigData_Factor10", "CatBoost_BigData_Factor10", "LightGBM_BigData_Factor10", "MLP_Classifier_BigData_Factor10",
               "XGBoost_BigData_Factor50", "RandomForest_BigData_Factor50", "CatBoost_BigData_Factor50", "LightGBM_BigData_Factor50", "MLP_Classifier_BigData_Factor50",
               "XGBoost_BigData_Factor100", "RandomForest_BigData_Factor100", "CatBoost_BigData_Factor100", "LightGBM_BigData_Factor100", "MLP_Classifier_BigData_Factor100"]

stacking_df = spark.table("my_catalog.default.stacking_bigdata_metrics").toPandas()
all_models_df = spark.table("my_catalog.default.model_efficiency_bigdata_tests").toPandas()

stacking_rows = []
for factor in [10, 50, 100]:
    stacking_row = stacking_df[stacking_df["factor"] == factor].iloc[0]
    base_rows = all_models_df[(all_models_df["factor"] == factor) & (all_models_df["model"].str.contains("XGBoost|RandomForest|CatBoost|LightGBM|MLP_Classifier"))]
    total_inference_time = base_rows["inference_time_sec"].sum() + stacking_row["inference_time_sec"]
    max_memory_usage = max(base_rows["memory_usage_mb"].max(), stacking_row["memory_usage_mb"])
    max_cpu_percent = max(base_rows["cpu_percent"].max(), stacking_row["cpu_percent"])
    max_num_threads = max(base_rows["num_threads"].max(), stacking_row["num_threads"])
    stacking_rows.append({
        "model": f"StackingEnsemble_BigData_Factor{factor}",
        "inference_time_sec": total_inference_time,
        "memory_usage_mb": max_memory_usage,
        "cpu_percent": max_cpu_percent,
        "num_threads": max_num_threads,
        "factor": factor,
        "total_records": stacking_row["total_records"]
    })

stacking_realistic_df = pd.DataFrame(stacking_rows)
all_models_with_stacking_df = pd.concat([all_models_df, stacking_realistic_df], ignore_index=True)
display(all_models_with_stacking_df)

In [0]:
stacking_realistic_df = pd.DataFrame(stacking_rows)
all_models_with_stacking_df = pd.concat([all_models_df, stacking_realistic_df], ignore_index=True)
spark_df_with_stacking = spark.createDataFrame(all_models_with_stacking_df)
spark_df_with_stacking.write.mode("overwrite").saveAsTable("my_catalog.default.model_efficiency_with_stacking_big_data")

In [0]:
model_efficiency_with_stacking_big_data_df = spark.table("my_catalog.default.model_efficiency_with_stacking_big_data")
display(model_efficiency_with_stacking_big_data_df)

In [0]:
model_efficiency_with_stacking_big_data_df = spark.table("my_catalog.default.model_efficiency_with_stacking_big_data").toPandas()
final_model_metrics_with_stacking_10_runs_df = spark.table("my_catalog.default.final_model_metrics_with_stacking_10_runs").toPandas()

original_count = len(X_test_scaled)

predictive_metrics = final_model_metrics_with_stacking_10_runs_df.set_index("model").to_dict(orient="index")

def get_prefix(model_name):
    return model_name.split("_")[0].lower()

prefix_metrics = {}
for model, metrics in predictive_metrics.items():
    prefix = get_prefix(model)
    prefix_metrics[prefix] = metrics

for col in ["roc_auc", "pr_auc", "precision", "recall", "f1_score", "weighted_score"]:
    model_efficiency_with_stacking_big_data_df[col] = model_efficiency_with_stacking_big_data_df["model"].apply(
        lambda x: prefix_metrics.get(get_prefix(x), {}).get(col, None)
    )

max_inference_time = 5.0
max_memory_usage = 1024.0
max_cpu_percent = 100.0
max_num_threads = 16.0

def compute_efficiency_score(row):
    inference_time = row["inference_time_sec"]
    memory_usage_mb = row["memory_usage_mb"]
    cpu_percent = row["cpu_percent"]
    num_threads = row["num_threads"]
    score = (
        (1 - min(inference_time / max_inference_time, 1)) * 0.4 +
        (1 - min(memory_usage_mb / max_memory_usage, 1)) * 0.4 +
        (1 - min(cpu_percent / max_cpu_percent, 1)) * 0.1 +
        (1 - min(num_threads / max_num_threads, 1)) * 0.1
    )
    return score

model_efficiency_with_stacking_big_data_df["efficiency_score"] = model_efficiency_with_stacking_big_data_df.apply(compute_efficiency_score, axis=1)
model_efficiency_with_stacking_big_data_df["overall_score"] = (
    model_efficiency_with_stacking_big_data_df["weighted_score"] * 0.7 +
    model_efficiency_with_stacking_big_data_df["efficiency_score"] * 0.3
)

final_model_metrics_with_stacking_10_runs_df["factor"] = 1
final_model_metrics_with_stacking_10_runs_df["total_records"] = original_count
cols_to_add = ["model", "inference_time_sec", "memory_usage_mb", "cpu_percent", "num_threads", "factor", "total_records",
               "roc_auc", "pr_auc", "precision", "recall", "f1_score", "weighted_score", "efficiency_score", "overall_score"]

for col in cols_to_add:
    if col not in final_model_metrics_with_stacking_10_runs_df.columns:
        final_model_metrics_with_stacking_10_runs_df[col] = None

df_final = pd.concat([model_efficiency_with_stacking_big_data_df[cols_to_add], final_model_metrics_with_stacking_10_runs_df[cols_to_add]], ignore_index=True)
df_final = df_final.sort_values("model", ascending=True).reset_index(drop=True)

spark_df_with_scores = spark.createDataFrame(df_final)
display(spark_df_with_scores)

In [0]:
df_scores = spark.table("my_catalog.default.model_efficiency_with_scores").toPandas()

In [0]:
df_scores = df_scores.drop(columns=[col for col in ["model_prefix", "modelo_base"] if col in df_scores.columns])
df_scores["model_name"] = df_scores["model"].apply(
    lambda x: "MLPClassifier" if x.lower().startswith("mlp") else ("LogisticRegression" if x.lower().startswith("logistic") else x.split("_")[0])
)
display(df_scores)

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

factors = sorted(df_scores["factor"].unique(), reverse=True)
model_name = df_scores["model_name"].unique()

pivot_df = df_scores.pivot_table(index="model_name", columns="factor", values="efficiency_score")

plt.figure(figsize=(18, 10))
plt.gca().set_facecolor('#f7f7f7')


ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)


bar_width = 0.25  
space_between_bars = 0.1  
group_spacing = 0.6  


colors = plt.cm.tab10(np.linspace(0, 1, len(factors)))


y_pos = np.arange(len(modelo_bases)) * (1 + group_spacing)


for idx, factor in enumerate(factors):
    scores = pivot_df[factor].values
    x_pos = y_pos + idx * (bar_width + space_between_bars)
    plt.barh(
        x_pos,
        scores,
        height=bar_width,
        color=colors[idx],
        alpha=0.85,
        label=f"Fator {factor}"
    )
    for i, score in enumerate(scores):
        if not np.isnan(score):
            
            if idx < len(factors) - 1:  
                prev_score = pivot_df[factors[idx + 1]].values[i]
                variation = ((score - prev_score) / prev_score) * 100
                label = f"{score:.3f} ({variation:+.1f}%)"
            else:
                label = f"{score:.3f}"
            plt.text(
                score,
                x_pos[i],
                label,
                va='center',
                ha='left',
                fontsize=10,
                color=colors[idx],
                fontweight='bold'
            )

plt.yticks(y_pos + (len(factors) - 1) * (bar_width + space_between_bars) / 2, model_name, fontsize=12)

plt.xlabel("\nScore de Eficiência Computacional", fontsize=10)
plt.title("\n Score de Eficiência Computacional por Modelo e Volume de Dados\n", fontweight='bold', fontsize=12)

plt.legend(
    title="Fator de Dados",
    loc="center left",
    bbox_to_anchor=(1, 0.5),
    fontsize=10,
    title_fontsize=12
)

plt.grid(axis='x', linestyle='--', alpha=0.3)
plt.tight_layout(rect=[0, 0, 0.85, 1])

plt.show()

In [0]:
summary = []
for model in pivot_df.index:
    score_1 = pivot_df.loc[model, 1] if 1 in pivot_df.columns else np.nan
    score_100 = pivot_df.loc[model, 100] if 100 in pivot_df.columns else np.nan
    if not np.isnan(score_1) and not np.isnan(score_100):
        variation = ((score_100 - score_1) / score_1) * 100
        summary.append({
            "Modelo": model,
            "Score Fator 1": f"{score_1:.3f}",
            "Score Fator 100": f"{score_100:.3f}",
            "Variação (%)": f"{variation:+.1f}",
            "var_num": variation 
        })

summary_df = pd.DataFrame(summary)
summary_df = summary_df.sort_values("var_num", ascending=True).drop(columns="var_num").reset_index(drop=True)
display(summary_df)

**Score de Eficiência Computacional por Modelo e Volume de Dados**
| Modelo            | Score Fator 1 | Score Fator 100 | Variação (%) |
|-------------------|--------------|-----------------|--------------|
| MLPClassifier     | 0.536        | 0.062           | -88.4        |
| LightGBM          | 0.594        | 0.076           | -87.2        |
| StackingEnsemble  | 0.412        | 0.054           | -86.8        |
| CatBoost          | 0.603        | 0.086           | -85.7        |
| XGBoost           | 0.541        | 0.081           | -85.1        |
| RandomForest      | 0.559        | 0.087           | -84.5        |
| DecisionTree      | 0.600        | 0.309           | -48.5        |
| LogisticRegression| 0.567        | 0.353           | -37.7        |

**Score de Eficiência Computacional por Modelo e Volume de Dados**
| Modelo            | Score Fator 1 | Score Fator 100 | Variação (%) |
|-------------------|--------------|-----------------|--------------|
| LogisticRegression| 0.567        | 0.353           | -37.7        |
| DecisionTree      | 0.600        | 0.309           | -48.5        |
| RandomForest      | 0.559        | 0.087           | -84.5        |
| XGBoost           | 0.541        | 0.081           | -85.1        |
| CatBoost          | 0.603        | 0.086           | -85.7        |
| StackingEnsemble  | 0.412        | 0.054           | -86.8        |
| LightGBM          | 0.594        | 0.076           | -87.2        |
| MLPClassifier     | 0.536        | 0.062           | -88.4        |

In [0]:
best_by_factor = []S
for factor in sorted(pivot_df.columns):
    best_model = pivot_df[factor].idxmax()
    best_score = pivot_df[factor].max()
    best_by_factor.append({
        "Fator": factor,
        "Melhor Modelo": best_model,
        "Score Ef. Comp.": f"{best_score:.3f}"
    })

best_by_factor_df = pd.DataFrame(best_by_factor)

mean_scores = pivot_df.mean(axis=1)
top3 = mean_scores.sort_values(ascending=False).head(3)
top3_df = pd.DataFrame({
    "Top 3 Modelos Gerais": top3.index,
    "Score Ef. Comp. Médio Geral": top3.map(lambda x: f"{x:.3f}").values
})

display(best_by_factor_df)
display(top3_df)

**Score Médio Geral de Eficiência Computacional por Modelo**
| Ranking | Modelo             | Score Eficiência Médio | Análise                          |
|---------|--------------------|-----------------------|----------------------------------|
| 1º      | LogisticRegression | 0.474                 | Melhor escalabilidade geral      |
| 2º      | DecisionTree       | 0.465                 | Equilíbrio eficiência-velocidade |
| 3º      | CatBoost           | 0.325                 | Líder entre ensemble models      |
| 4º      | LightGBM           | 0.300                 | Eficiência moderada              |
| 5º      | RandomForest       | 0.225                 | Custo de bagging evidente        |
| 6º      | MLPClassifier      | 0.220                 | Redes neurais ineficientes       |
| 7º      | XGBoost            | 0.219                 | Alta complexidade penaliza       |
| 8º      | StackingEnsemble   | 0.165                 | Maior custo computacional        |

In [0]:
pivot_overall = df_scores.pivot_table(index="model_name", columns="factor", values="overall_score")

summary_overall = []
for model in pivot_overall.index:
    score_1 = pivot_overall.loc[model, 1] if 1 in pivot_overall.columns else np.nan
    score_100 = pivot_overall.loc[model, 100] if 100 in pivot_overall.columns else np.nan
    if not np.isnan(score_1) and not np.isnan(score_100):
        variation = ((score_100 - score_1) / score_1) * 100
        summary_overall.append({
            "Modelo": model,
            "Trade-off Fator 1": f"{score_1:.3f}",
            "Trade-off Fator 100": f"{score_100:.3f}",
            "Variação (%)": f"{variation:+.1f}",
            "var_num": variation
        })

summary_overall_df = pd.DataFrame(summary_overall)
summary_overall_df = summary_overall_df.sort_values("var_num", ascending=True).drop(columns="var_num").reset_index(drop=True)
display(summary_overall_df)

In [0]:
pivot_overall = df_scores.pivot_table(index="model_name", columns="factor", values="overall_score")

best_by_factor = []
for factor in sorted(pivot_overall.columns):
    best_model = pivot_overall[factor].idxmax()
    best_score = pivot_overall[factor].max()
    best_by_factor.append({
        "Fator": factor,
        "Melhor Modelo": best_model,
        "Score Trade-off": f"{best_score:.3f}"
    })

best_by_factor_df = pd.DataFrame(best_by_factor)

mean_scores = pivot_overall.mean(axis=1)
top3 = mean_scores.sort_values(ascending=False).head(3)
top3_df = pd.DataFrame({
    "Top 3 Modelos Gerais": top3.index,
    "Score Trade-off Médio Geral": top3.map(lambda x: f"{x:.3f}").values
})

display(best_by_factor_df)
display(top3_df)

**Volume Base (Fator 1):**
| Ranking | Modelo             | Score Desempenho Preditivo | Score Ef. Comp. | Trade-off |
|---------|--------------------|---------------|------------------|--------------|
| 1º      | CatBoost           | 0.848         | 0.603            | 0.775        |
| 2º      | RandomForest       | 0.857         | 0.559            | 0.768        |
| 3º      | XGBoost            | 0.861         | 0.541            | 0.765        |
| 4º      | LightGBM           | 0.838         | 0.594            | 0.765        |
| 5º      | MLP_Classifier     | 0.842         | 0.536            | 0.750        |
| 6º      | LogisticRegression | 0.820         | 0.567            | 0.744        |
| 7º      | DecisionTree       | 0.815         | 0.600            | 0.751        |
| 8º      | StackingEnsemble   | 0.842         | 0.412            | 0.713        |

**Volume Escalado (Fator 100):**
| Ranking | Modelo             | Trade-off | Variação vs Fator 1 |
|---------|--------------------|--------------|---------------------|
| 1º      | LogisticRegression | 0.680        | -8.6%               |
| 2º      | DecisionTree       | 0.663        | -11.6%              |
| 3º      | XGBoost            | 0.627        | -18.1%              |
| 4º      | RandomForest       | 0.626        | -18.5%              |
| 5º      | CatBoost           | 0.620        | -20.0%              |
| 6º      | LightGBM           | 0.609        | -20.3%              |
| 7º      | MLPClassifier      | 0.608        | -18.9%              |
| 8º      | StackingEnsemble   | 0.606        | -15.0%              |

In [0]:
#import matplotlib.pyplot as plt
#import numpy as np

mean_efficiency = df_scores.groupby("model_name")["efficiency_score"].mean().sort_values(ascending=False)
mean_overall = df_scores.groupby("model_name")["overall_score"].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
bars = plt.bar(mean_efficiency.index, mean_efficiency.values, color='#87ceeb')
plt.title("\nScore Médio Geral de Eficiência Computacional por Modelo\n", fontweight='bold')
plt.xticks(rotation=45)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().axes.get_yaxis().set_visible(False)
plt.ylabel("")
plt.xlabel("Modelo")
plt.tight_layout()
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{height:.4f}", ha='center', va='bottom', fontsize=10)
plt.show()

plt.figure(figsize=(10, 6))
bars = plt.bar(mean_overall.index, mean_overall.values, color='#90ee90')
plt.title("\nScore Médio Geral de Trade-off (Desempenho x Eficiência) por Modelo\n", fontweight='bold')
plt.xticks(rotation=45)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().axes.get_yaxis().set_visible(False)
plt.ylabel("")
plt.xlabel("Modelo")
plt.tight_layout()
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{height:.4f}", ha='center', va='bottom', fontsize=10)
plt.show()